In [ ]:
import cv2
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load and pre-process image data
def load_data():
    # Load images and labels
    images = []
    labels = []
    for i in range(1, 25):
        image = cv2.imread("obj_img/obj(" + str(i) + ").jpg")
        images.append(image)
        labels.append(1)
    for i in range(1, 11):
        image = cv2.imread("obj_def_img/def_obj(" + str(i) + ").jpg")
        images.append(image)
        labels.append(0)
    return images, labels

In [ ]:
# Extract features from image using OpenCV
def extract_features(images):
    features = []
    for image in images:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        features.append(gray.ravel())
    return features
# Train and test SVM model

In [ ]:
def train_test_model(features, labels):
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
    
    #scaling the data
    sc=StandardScalar()
    sc.fit(x_train)
    x_train_sc=sc.transform(X_train)
    x_test_sc=sc.transform(X_test)
    y_train_sc=sc.transform(y_train)
    y_test_sc=sc.transform(y_test)
    
    clf = svm.SVC(kernel='linear', C=1)
    clf.fit(X_train, y_train)
    accuracy = clf.score(X_test, y_test)
    print("Accuracy: ", accuracy)

In [ ]:
# Predict if input image is defective or not
def predict(image, clf):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    feature = gray.ravel()
    prediction = clf.predict([feature])
    if prediction[0] == 1:
        print("The object is good.")
    else:
        print("The object is defective.")

In [ ]:
#using the camera for live prediction
def live_predict_defect(clf):
    cap = cv2.VideoCapture(0)

    while(True):
        ret, frame = cap.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        #frame = cv2.resize(frame, None, fx = scale_factor, fy = scale_factor, interpolation = cv2.INTER_AREA)
        #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        feature = gray.ravel()
        prediction = clf.predict([feature])
        if prediction[0] == 1:
            cv2.putText(frame, "GOOD", (20,30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        else:
            cv2.putText(frame, "DEFECTIVE", (20,30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.imshow("Live Quality Control", frame)

        if cv2.waitKey(0) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()



In [ ]:
# Main function
if __name__ == '__main__':
    images, labels = load_data()
    features = extract_features(images)
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
    clf = svm.SVC(kernel='linear', C=1)
    clf.fit(X_train, y_train)
    accuracy = clf.score(X_test, y_test)
    print("Accuracy: ", accuracy)
    # assuming the classifier is already trained and stored in the 'clf' variable
    live_predict_defect(clf)

    
    # Take input image - Ask for input image path
#    img_path = input("Enter the path of the image: ")
#    input_image = cv2.imread(img_path)
#    predict(input_image, clf)
#    cv2.namedWindow("Input Image", cv2.WINDOW_NORMAL)
#    cv2.resizeWindow("Input Image", 360, 640)
#    cv2.imshow("Input Image", input_image)
#    cv2.waitKey(0)
#    cv2.destroyAllWindows()